**Question 1: Value Iteration Algorithm**

In [4]:
import numpy as np

# Parameters
gamma = 0.9  # Discount factor for future rewards
theta = 1e-6  # Convergence threshold for value iteration
grid_size = 5  # Define the size of the grid world

# Initialize rewards and value function
rewards = np.full((grid_size, grid_size), -1)  # Default reward of -1 for every step
rewards[-1, -1] = 10  # Goal state reward of +10
V = np.zeros((grid_size, grid_size))  # Value function initialized to zero

# Actions: (dx, dy) format
actions = {"up": (-1, 0), "down": (1, 0), "left": (0, -1), "right": (0, 1)}

In [5]:
# Value Iteration Algorithm
def value_iteration():
    """Performs value iteration to find the optimal value function."""
    global V
    while True:
        delta = 0  # Track maximum change in value function
        new_V = np.copy(V)
        for i in range(grid_size):
            for j in range(grid_size):
                if (i, j) == (grid_size-1, grid_size-1):
                    continue  # Skip goal state since its value is fixed

                max_value = float('-inf')  # Store best action value
                for action, (di, dj) in actions.items():
                    ni, nj = i + di, j + dj  # Next state
                    if 0 <= ni < grid_size and 0 <= nj < grid_size:
                        value = rewards[i, j] + gamma * V[ni, nj]  # Bellman update
                        max_value = max(max_value, value)
                new_V[i, j] = max_value
                delta = max(delta, abs(new_V[i, j] - V[i, j]))  # Track max change
        V = new_V
        if delta < theta:  # Stop when convergence is achieved
            break


# Run value iteration
value_iteration()
print("Optimal Value Function:")
print(V)

Optimal Value Function:
[[-5.6953279 -5.217031  -4.68559   -4.0951    -3.439    ]
 [-5.217031  -4.68559   -4.0951    -3.439     -2.71     ]
 [-4.68559   -4.0951    -3.439     -2.71      -1.9      ]
 [-4.0951    -3.439     -2.71      -1.9       -1.       ]
 [-3.439     -2.71      -1.9       -1.         0.       ]]


In [6]:
# Extract policy
def extract_policy():
    """Extracts the optimal policy from the value function."""
    policy = np.empty((grid_size, grid_size), dtype=object)
    for i in range(grid_size):
        for j in range(grid_size):
            if (i, j) == (grid_size-1, grid_size-1):
                policy[i, j] = 'G'  # Mark goal state
                continue

            best_action = None
            best_value = float('-inf')
            for action, (di, dj) in actions.items():
                ni, nj = i + di, j + dj
                if 0 <= ni < grid_size and 0 <= nj < grid_size:
                    value = rewards[i, j] + gamma * V[ni, nj]
                    if value > best_value:
                        best_value = value
                        best_action = action[0].upper()  # Store best action
            policy[i, j] = best_action
    return policy

policy = extract_policy()
print("Optimal Policy:")
print(policy)

Optimal Policy:
[['D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'D' 'D']
 ['D' 'D' 'D' 'D' 'D']
 ['R' 'R' 'R' 'R' 'G']]


**Question 2: Q-Learning Algorithm**

In [7]:
# Q-Learning Algorithm
alpha = 0.5  # Learning rate
epsilon = 0.1  # Exploration factor
episodes = 5000  # Number of training episodes
Q = np.zeros((grid_size, grid_size, len(actions)))  # Initialize Q-table with zeros

# Convert action names to indices
action_list = list(actions.keys())

In [8]:
def choose_action(state):
    """Chooses an action using the epsilon-greedy strategy."""
    if np.random.rand() < epsilon:
        return np.random.choice(len(actions))  # Explore: random action
    return np.argmax(Q[state[0], state[1]])  # Exploit: best known action

def q_learning():
    """Performs Q-learning to find the optimal action-value function."""
    for _ in range(episodes):
        state = (0, 0)  # Start at top-left corner
        while state != (grid_size-1, grid_size-1):  # Until reaching goal state
            action_idx = choose_action(state)  # Choose an action
            action = action_list[action_idx]
            di, dj = actions[action]  # Get action direction
            new_state = (state[0] + di, state[1] + dj)
            if not (0 <= new_state[0] < grid_size and 0 <= new_state[1] < grid_size):
                new_state = state  # Stay in place if out of bounds

            reward = rewards[state[0], state[1]]  # Get reward for current state
            best_next_q = np.max(Q[new_state[0], new_state[1]])  # Best future Q-value

            # Q-learning update rule
            Q[state[0], state[1], action_idx] += alpha * (reward + gamma * best_next_q - Q[state[0], state[1], action_idx])

            state = new_state  # Move to next state

# Run Q-learning
q_learning()
print("Learned Q-Values:")
print(Q)

Learned Q-Values:
[[[-6.12579511 -5.6953279  -6.12579511 -5.6953279 ]
  [-5.69501728 -5.217031   -6.10014895 -5.217031  ]
  [-5.04180508 -4.68559    -5.65081423 -4.68559   ]
  [-4.65663594 -4.0951     -5.17034904 -4.0951    ]
  [-3.81774001 -3.439      -4.27823847 -4.03083014]]

 [[-6.12579511 -5.217031   -5.6953279  -5.217031  ]
  [-5.69524117 -4.68559    -5.68593936 -4.68559   ]
  [-4.93652236 -4.0951     -4.88028184 -4.0951    ]
  [-4.67081451 -3.439      -4.42501667 -3.439     ]
  [-3.85592925 -2.71       -3.9520281  -3.38254162]]

 [[-5.6953279  -4.68559    -5.217031   -4.68559   ]
  [-5.21596547 -4.0951     -5.2169172  -4.0951    ]
  [-4.56703604 -3.439      -4.61394308 -3.439     ]
  [-4.05661431 -2.71       -3.81893422 -2.71      ]
  [-3.18356468 -1.9        -3.43815674 -2.6927417 ]]

 [[-5.217031   -4.0951     -4.68559    -4.0951    ]
  [-4.68558158 -3.439      -4.68558692 -3.439     ]
  [-4.09345105 -2.71       -4.09349181 -2.71      ]
  [-3.43375367 -1.9        -3.42952386 -